# Convert Crons to Human Readable strings

cron-descriptor python library: https://pypi.org/project/cron-descriptor/


## Goal: Create a Snowpark UDF that converts chron spec to human readable string that can be called in Sigma

_____________


#### Precursor Steps:

1. Follow along with the quickstarts: https://quickstarts.sigmacomputing.com/guide/snowflake_snowpark_udf_python/index.html?index=..%2F..index#0
2. Most importantly, pay attention to the virtual environment section.
3. ```pip install cron-descriptor```


_____________

### 1. Enter username and password for cxa

In [67]:
# Change all
snowflake_account = "account" # ex: "cxa..."
dev_username = "my username"
password = "my password"
role = "my role"
warehouse = "my warehouse"
database = "my db"
schema = "my schema"

#### 2. Configure Snowpark connection

In [23]:
import numpy as np
import pandas as pd

from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

connection_parameters = {
    "account": snowflake_account,
    "user": dev_username,
    "password": password,
    "role": role,  # optional
    "warehouse": warehouse,  # optional
    "database": database,  # optional
    "schema": schema,  # optional
}  

session = Session.builder.configs(connection_parameters).create()

#### 3. Create a UDF that contains the actual logic. Grant all on the function for the correct role so it can be used in Sigma.

In [66]:
# Register this as vectorized function that we can reference in Sigma

def cron_descriptor_udf(cron_input: str) -> dict:

    from cron_descriptor import get_description
    return {'cron_input': cron_input, 'cron_readable': get_description(cron_input)}


# Register Vectorized UDF
cron_descriptor_udf = session.udf.register(func=cron_descriptor_udf,
                                                        name="CRON_DESCRIPTOR_UDF",
                                                        stage_location='@DEMENG_ML_MODELS', # can point to any stage in the schema, doesn't mattter which but needs to be included
                                                        input_types=[T.StringType()],
                                                        return_type=T.VariantType(),
                                                        replace=True,
                                                        is_permanent=True,
                                                        packages=['pandas', 'cron-descriptor'],
                                                        max_batch_size=1000,
                                                        session=session)

query = session.sql('grant all on function CRON_DESCRIPTOR_UDF(varchar) to role PAPERCRANE')
query.collect()

The version of package cron-descriptor in the local environment is 1.2.35, which does not fit the criteria for the requirement cron-descriptor. Your UDF might not work when the package version is different between the server and your local environment


[Row(status='Statement executed successfully.')]